## Import modules and define constants

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from wrf import interplevel, getvar, destagger
from matplotlib.cm import get_cmap
CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.

## Define user parameters

In [ ]:
dx = 30000.
dy = 30000.
nx = 500
ny = 240
nz_coamps = 45
nz_wrf = 100
shape_coamps = [nz_coamps, ny, nx]
shape_wrf = [nz_wrf, ny, nx]

wrf_dir = '/p/work1/lloveras/third_adj/in_files/wrfin_altadj'
wrfout_dir = '/p/work1/lloveras/third_adj/out_files/wrfout_ctl'
coamps_dir = '/p/work1/lloveras/third_adj/sens_0h/'
full_dir = '/p/work1/lloveras/third_adj/pert_0h/'

dz_coamps = np.array([20, 20, 30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 
                      380, 400, 430, 470, 500, 500, 500, 550, 600, 650, 700, 750, 750, 750, 750, 750, 750, 750, 750,
                      800, 900, 900, 900, 900, 900])

## Read in data

In [ ]:
ncfile = Dataset(wrf_dir,'r+')
ncfile_wrfout = Dataset(wrfout_dir)
wrf_u = np.array(getvar(ncfile,'U'))
wrf_v = np.array(getvar(ncfile,'V'))
wrf_p = np.array(getvar(ncfile,'P'))
wrf_th = np.array(getvar(ncfile,'T'))
wrf_qv = np.array(getvar(ncfile,'QVAPOR'))
wrf_full_p = np.array(getvar(ncfile,'pressure',timeidx=0))

sens_u = np.fromfile(coamps_dir + 'aaauu4_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')
sens_v = np.fromfile(coamps_dir + 'aaavv4_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')
sens_ex = np.fromfile(coamps_dir + 'aaapp4_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')
sens_th = np.fromfile(coamps_dir + 'aaath4_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')
sens_qv = np.fromfile(coamps_dir + 'aaaqv4_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')
coamps_full_p = np.fromfile(full_dir + 'ttlprs_sig_020000_000010_1a0500x0240_2021060100_00000000_fcstfld', dtype='>f4')

sens_u = np.flip(np.reshape(sens_u, shape_coamps),axis=0)
sens_v = np.flip(np.reshape(sens_v, shape_coamps),axis=0)
sens_ex = np.flip(np.reshape(sens_ex, shape_coamps),axis=0)
sens_th = np.flip(np.reshape(sens_th, shape_coamps),axis=0)
sens_qv = np.flip(np.reshape(sens_qv, shape_coamps),axis=0)
coamps_full_p = np.flip(np.reshape(coamps_full_p, shape_coamps),axis=0)
coamps_full_ex = (coamps_full_p/P0)**(RD/CP)

for i in range(nx):
    for j in range(ny):
        if (i <= 10 or i >= 490) or (j <= 10 or j >= 230):
            sens_u[:,j,i] = np.zeros(nz_coamps)
            sens_v[:,j,i] = np.zeros(nz_coamps)
            sens_ex[:,j,i] = np.zeros(nz_coamps)
            sens_th[:,j,i] = np.zeros(nz_coamps)
            sens_qv[:,j,i] = np.zeros(nz_coamps)

## Alternative perturbations

In [ ]:
du = np.array(getvar(ncfile_wrfout,'ua',timeidx=12)) - np.array(getvar(ncfile_wrfout,'ua',timeidx=0))
dv = np.array(getvar(ncfile_wrfout,'va',timeidx=12)) - np.array(getvar(ncfile_wrfout,'va',timeidx=0))
dth = np.array(getvar(ncfile_wrfout,'th',timeidx=12)) - np.array(getvar(ncfile_wrfout,'th',timeidx=0))
dqv = np.array(getvar(ncfile_wrfout,'QVAPOR',timeidx=12)) - np.array(getvar(ncfile_wrfout,'QVAPOR',timeidx=0))
p2 = np.array(getvar(ncfile_wrfout,'pressure',timeidx=12))
p1 = np.array(getvar(ncfile_wrfout,'pressure',timeidx=0))
ex2 = ((p2/P0)**(RD/CP))
ex1 = ((p1/P0)**(RD/CP))
dex = ex2 - ex1

w_u = np.amax(np.abs(du))**(-2)
w_v = np.amax(np.abs(dv))**(-2)
w_th = np.amax(np.abs(dth))**(-2)
w_ex = np.amax(np.abs(dex))**(-2)
w_qv = np.amax(np.abs(dqv))**(-2)

In [ ]:
s = 1.54e-6
u_pert = np.zeros(shape_coamps)
v_pert = np.zeros(shape_coamps)
ex_pert = np.zeros(shape_coamps)
th_pert = np.zeros(shape_coamps)
qv_pert = np.zeros(shape_coamps)
for k in range(nz_coamps):
    u_pert[k,:,:] = sens_u[k,:,:]*dz_coamps[k]*s/w_u
    v_pert[k,:,:] = sens_v[k,:,:]*dz_coamps[k]*s/w_v
    ex_pert[k,:,:] = sens_ex[k,:,:]*dz_coamps[k]*s/w_ex
    th_pert[k,:,:] = sens_th[k,:,:]*dz_coamps[k]*s/w_th
    qv_pert[k,:,:] = sens_qv[k,:,:]*dz_coamps[k]*s/w_qv
    
full_ex = coamps_full_ex + ex_pert
full_p = P0*full_ex**(CP/RD)
p_pert = (full_p - coamps_full_p)*100.

In [ ]:
umax = np.amax(np.abs(u_pert))
vmax = np.amax(np.abs(v_pert))
thmax = np.amax(np.abs(th_pert))
pmax = np.amax(np.abs(p_pert))
qvmax = np.amax(np.abs(qv_pert))
print(umax, vmax, thmax, pmax, qvmax*1000)

In [ ]:
u_pert_wrf = np.zeros([nz_wrf,ny,nx+1])
v_pert_wrf = np.zeros([nz_wrf,ny+1,nx])
p_pert_wrf = np.zeros(shape_wrf)
th_pert_wrf = np.zeros(shape_wrf)
qv_pert_wrf = np.zeros(shape_wrf)
for k in range(nz_wrf):
    u_pert_wrf[k,:,:-1] = interplevel(u_pert, coamps_full_p, wrf_full_p[k,:,:])
    v_pert_wrf[k,:-1,:] = interplevel(v_pert, coamps_full_p, wrf_full_p[k,:,:])
    th_pert_wrf[k,:,:] = interplevel(th_pert, coamps_full_p, wrf_full_p[k,:,:])
    p_pert_wrf[k,:,:] = interplevel(p_pert, coamps_full_p, wrf_full_p[k,:,:])
    qv_pert_wrf[k,:,:] = interplevel(qv_pert, coamps_full_p, wrf_full_p[k,:,:])
    
u_pert_wrf[:,:,-1] = u_pert_wrf[:,:,-2]
v_pert_wrf[:,-1,:] = v_pert_wrf[:,-2,:]

u_pert_wrf = np.nan_to_num(u_pert_wrf)
v_pert_wrf = np.nan_to_num(v_pert_wrf)
th_pert_wrf = np.nan_to_num(th_pert_wrf)
p_pert_wrf = np.nan_to_num(p_pert_wrf)
qv_pert_wrf = np.nan_to_num(qv_pert_wrf)

In [ ]:
ncfile.variables['U'][0,:,:,:] = wrf_u + u_pert_wrf
ncfile.variables['V'][0,:,:,:] = wrf_v + v_pert_wrf
ncfile.variables['T'][0,:,:,:] = wrf_th + th_pert_wrf
ncfile.variables['P'][0,:,:,:] = wrf_p + p_pert_wrf
ncfile.variables['QVAPOR'][0,:,:,:] = wrf_qv + qv_pert_wrf

ncfile.close()